# BITS F312 - Neural Network and Fuzzy Logic



# NNFL Assignment 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL/Data_A2/

/content/drive/MyDrive/NNFL/Data_A2


In [ ]:
# listing datasets
%ls -l

total 1234719
-rw------- 1 root root     637638 Oct 31 04:55 Assignment2.pdf
-rw------- 1 root root        259 Oct 31 04:57 class_label.mat
-rw------- 1 root root      40295 Oct 31 04:57 data55.xlsx
-rw------- 1 root root      21269 Oct 31 04:55 data5.xlsx
-rw------- 1 root root 1263647365 Oct 31 04:58 input.mat
drwx------ 2 root root       4096 Nov 18 05:58 logs/


In [ ]:
# libraries required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
# supressing warnings
import warnings
warnings.filterwarnings('ignore')

# Q4
Implement the radial basis function neural network (RBFNN) for the classification problem. You can use
Gaussian, multiquadric and linear kernel functions for the implementation. You can use both holdout (70,
10, and 20%) and 5-fold cross-validation approaches for evaluating the performance of the classifier. The
classification performance must be evaluated using individual accuracy and overall accuracy measures.
The dataset (data5.xlsx) contains 7 features and the last column is the output (class labels). (Packages
such as Scikitlearn, keras, tensorflow, pytorch etc. are not allowed).

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

def kmeansClustering(k):
    kmeans = KMeans(n_clusters = k).fit(train_x)
    return kmeans

def kernel_func(x, std, sigma, kernalFunction):
    beta = 1/(2*sigma*sigma)
    if kernalFunction == "Gaussian":
        return np.exp(-beta*(np.linalg.norm(x-std))**2)
    elif kernalFunction == "MultiQuadratic":
        return ((np.linalg.norm(x-std))**2+sigma**2)**0.5
    elif kernalFunction == "Linear":
        return np.linalg.norm(x-std)

def sigmaComputation(x, labels, std):
    c = std.shape[0]
    sigma = np.zeros(c)
    for i in range(c):
        x_temp = x[labels==i]
        k = 0
        for j in range(x_temp.shape[0]):
            k+=np.linalg.norm(x_temp[j]-std[i])
        sigma[i] = k/x_temp.shape[0]
    return sigma

def hypothesisCalc(X, std, sigma, kernalFunction):
    c = std.shape[0]
    H = np.zeros((X.shape[0],c))
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            H[i][j]=kernel_func(X[i],std[j],sigma[j],kernalFunction)
    return H

def compute(train_x, train_y, test_x, test_y, kernalFunction):
    kmeans = kmeansClustering(20)
    std = kmeans.cluster_centers_
    sigma = sigmaComputation(train_x,kmeans.labels_,std)
    H = hypothesisCalc(train_x,std,sigma,kernalFunction)
    W = np.dot(np.linalg.pinv(H),train_y)
    H = hypothesisCalc(test_x,std,sigma,kernalFunction)
    pred = np.dot(H,W)
    p = (pred>0.5).astype(int)
    a = (p!=test_y).astype(int)
    metrics(test_y, pred_y) 


In [ ]:
def resultQ4(filename = 'data5.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = training_data[:, :feats]
  training_data_y = training_data[:, feats]
  validation_data_X = validation_data[:, :feats]
  validation_data_y = validation_data[:, feats]
  testing_data_X = testing_data[:, :feats]
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  print('Linear Kernel')
  compute(training_data_X, training_data_y, testing_data_X, testing_data_y, "Linear")
  compute(training_data_X, training_data_y, validation_data_X, validation_data_y, "Linear")

  print('Multi Quadratic Kernel')
  compute(training_data_X, training_data_y, testing_data_X, testing_data_y, "MultiQuadratic")
  compute(training_data_X, training_data_y, validation_data_X, validation_data_y, "MultiQuadratic")


In [ ]:
resultQ4()

Linear Kernel
Testing Accuracy: 

Sensitivity :  0.8461538461538461
Specificity :  0.7209302325581395
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.7804878048780488
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.8620689655172413
Specificity :  0.9183673469387755
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8878504672897196
---------------------------------------------------------------------------
Multi Quadratic Kernel
Testing Accuracy: 

Sensitivity :  0.7868852459016393
Specificity :  0.8636363636363636
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8267716535433071
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.8837209302325582
Specificity :  0.803921568627451
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8404255319148937
---------------------------------------------------------------------------
